# SQL Operations Demo

One of the benefits of Spark is its support for SQL operations when it comes to streaming data. 

To do this, one first creates a `SparkSession` using the `SparkContext` that the `StreamingContext` is using. 


### Demo

For this demonstration, we will be using Spark and SQL to analyze data from the [KDD Cup 1999 competition dataset](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99). The file is provided as a Gzip file that we will download locally.

In [ ]:
import urllib
from pyspark.sql import SQLContext
from pyspark.sql import Row

f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()
    

A Spark DataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R or Pandas. They can be constructed from a wide array of sources such as an existing RDD in our case.

The entry point into all SQL functionality in Spark is the SQLContext class. To create a basic instance, all we need is a SparkContext reference. Since we are running Spark in shell mode (using pySpark) we can use the global context object sc for this purpose.

In [ ]:
sqlContext = SQLContext(sc)


With a SQLContext, we are ready to create a DataFrame from our existing RDD. But first we need to tell Spark SQL the schema in our data.

Spark SQL can convert an RDD of Row objects to a DataFrame. Rows are constructed by passing a list of key/value pairs as kwargs to the Row class. The keys define the column names, and the types are inferred by looking at the first row. Therefore, it is important that there is no missing data in the first row of the RDD in order to properly infer the schema.

In our case, we first need to split the comma separated data, and then use the information in KDD's 1999 task description to obtain the column names.

In [ ]:
csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(duration=int(p[0]), protocol_type=p[1], service=p[2], flag=p[3],
                                      src_bytes=int(p[4]), dst_bytes=int(p[5])))


Once we have our RDD of Row we can infer and register the schema.

In [ ]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")


Now we can run SQL queries over our data frame that has been registered as a table. 

With the following SQL commands, we can select tcp network interactions with more than 1 second duration and no transfer from destination

In [ ]:
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0""")
tcp_interactions.show()


The results of SQL queries are RDDs and support all the normal RDD operations.

Next we create an Output duration together with `dst_bytes`.

In [ ]:
tcp_interactions_out = tcp_interactions.map(lambda p: "Duration: {}, Dest. bytes: {}".format(p.duration, p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
    print ti_out


We can easily have a look at our data frame schema using printSchema.

In [ ]:
interactions_df.printSchema()

You can also run SQL queries on tables defined on streaming data from a different thread, as long as you make sure that you set the StreamingContext to remember a sufficient amount of streaming data such that the query can run. If you don't provide enough data the StreamingContext will delete old streaming data before the query can complete. 

An example: if you want to query the last batch, but your query can take 5 minutes to run, try calling `streamingContext.remember(Minutes(5))`

## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#dataframe-and-sql-operations
2. https://spark.apache.org/docs/latest/sql-programming-guide.html